In [1]:
import numpy as np
import torch
from sklearn.decomposition import PCA
from abc import ABCMeta, abstractmethod

from bgan.datasets import SynthDataset
from synth_utils import js_div, kl_div, pca
from priors import FactorizedNormalPrior

from matplotlib import pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'bgan.datasets'; 'bgan' is not a package

In [2]:
import bgan

In [7]:
import sys

In [8]:
sys.path

['',
 '/Users/IzmailovPavel/Documents/Education/Projects/bayesganfuture/semisupervised_experiments',
 '/Users/IzmailovPavel/Documents/Education/Projects/t3f/t3f',
 '/Users/IzmailovPavel/Documents/Education/Projects/t3f',
 '/Users/IzmailovPavel/Documents/Education/Projects/Course Project/Code',
 '/Users/IzmailovPavel/Documents/Education/Projects/TTGP',
 '/Users/IzmailovPavel/Documents/Education/Projects/GP-SKI',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python36.zip',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages',
 '/Users/IzmailovPavel/Documents/Education/Projects/gym',
 '/Users/IzmailovPavel/Documents/Education/Projects/GPflow',
 '/Users/IzmailovPavel/Documents/Education/Projects/bayesganfuture/bgan',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/ext